In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import normalize
from sklearn.model_selection import train_test_split

In [2]:
Data_dir_train = "./sets/train"
CATEGORIES = ["NORMAL", "PNEUMONIA"]
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path_train = os.path.join(Data_dir_train, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path_train):
            try:
                img_array_train = cv2.imread(os.path.join(path_train, img), cv2.IMREAD_GRAYSCALE)
                img_array_resized = cv2.resize(img_array_train, (128, 128))
                training_data.append([img_array_resized, class_num])
            except Exception as e:
                print("broken img: ", img)
                pass
    random.shuffle(training_data)
    print(len(training_data))
    
create_training_data()        

5216


In [3]:
#check if labeling and shuffling of the data did accure
for sample in training_data[:10]:
    print(sample[1])

0
0
1
0
1
0
0
1
1
0


In [5]:
X = []
y = []
    
for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, 128, 128, 1)
y = np.array(y)

In [5]:
Data_dir_test = "./sets/test"
CATEGORIES = ["NORMAL", "PNEUMONIA"]
test_data = []

def create_testing_data():
    for category in CATEGORIES:
        path_test = os.path.join(Data_dir_test, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path_test):
            try:
                img_array_test = cv2.imread(os.path.join(path_test, img), cv2.IMREAD_GRAYSCALE)
                img_array_resized = cv2.resize(img_array_test, (128, 128))
                test_data.append([img_array_resized, class_num])
            except Exception as e:
                print("broken img %n", img)
                pass
    random.shuffle(test_data)
    print(len(test_data))
    
create_testing_data()        

639


In [6]:
for sample in test_data[:10]:
    print(sample[1])

0
1
1
1
0
0
1
1
0
1


In [7]:
X_UNSEEN = []
y_UNSEEN = []
    
for features, labels in test_data:
    X_UNSEEN.append(features)
    y_UNSEEN.append(labels)
        
X_UNSEEN = np.array(X_UNSEEN).reshape(-1, 128, 128, 1)
y_UNSEEN = np.array(y_UNSEEN)

In [8]:
#creating the model
recognizer = Sequential()
                                                                   
#Creating the first 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))                                      
                                         
#Creating the second 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))                                      
                                         
#Creating the third 2D convolutional and pooling layer
recognizer.add(Convolution2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='glorot_uniform', bias_initializer='zeros'))
recognizer.add(MaxPooling2D(pool_size=(2,2), padding='same'))
        
#Transforming the pooling layer into a vector where each vector vlue = neuron
recognizer.add(Flatten())
        
#Creating the Artificial Neural Network
recognizer.add(Dense(input_dim = 256, units=128, activation='relu'))
recognizer.add(Dense(input_dim =128, units= 64, activation='relu'))
recognizer.add(Dense(input_dim =64, units= 1, activation='sigmoid'))

#Compiling the Neural Network
recognizer.compile(optimizer = "adam" , loss="binary_crossentropy" , metrics=['accuracy'])

In [9]:
model_name = "3_conv2D_x_3_dense_256_128_64_adam"

In [10]:
tensorboard = tf.keras.callbacks.TensorBoard(
                        log_dir="{}\\".format(model_name),
                        histogram_freq=0,
                        write_graph=True,
                        write_images=False,
                        update_freq="epoch",
                        profile_batch=2,
                        embeddings_freq=0,
                        embeddings_metadata=None,
                )

In [15]:
X = X/255.0
X_UNSEEN = X_UNSEEN/255.0 

In [11]:
recognizer.fit(X, y, batch_size=32, validation_split=0.1, epochs=10, callbacks=[tensorboard])

TypeError: Value passed to parameter 'input' has DataType uint8 not in list of allowed values: float16, bfloat16, float32, float64, int32